<a href="https://colab.research.google.com/github/roydendsouza31/AI-Generated-Images-vs-Real-Images/blob/R3/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AI Generated Images vs Real Images

In [12]:
! pip install simple_image_download

INFO: pip is looking at multiple versions of simple-image-download to determine which version is compatible with other requirements. This could take a while.


In [13]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2 as cv
from simple_image_download import simple_image_download as simp
import random
import PIL
import pickle

In [20]:
#!rm -r simple_images

In [21]:
response = simp.simple_image_download
limit = 100
response().download("ai generated images", 100)

In [22]:
response().download("ai generated art", 100)

HTTPConnectionPool(host='no.access', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7b4ad28318d0>: Failed to resolve 'no.access' ([Errno -2] Name or service not known)"))


In [23]:
response().download("ai generated characters", 100)

In [25]:
response().download('stable diffusion', 100)

In [24]:
response().download('dalle2 generated images', 100)

In [26]:
response().download('midjourney', 100)

In [28]:
response().download('cityscapes', 100)

In [29]:
response().download('animals', 100)

In [30]:
response().download('vehicles', 50)

HTTPSConnectionPool(host='oshkoshdefense.com', port=443): Max retries exceeded with url: /wp-content/uploads/2022/07/201007-M-EM580-1191-Marines-1.jpg (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:1007)')))


In [31]:
response().download('traffic', 50)

HTTPSConnectionPool(host='www.progressive.com', port=443): Max retries exceeded with url: /lifelanes/wp-content/uploads/2023/09/TrafFicJam_Banner.jpg (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))


In [42]:
response().download('offices', 30)

In [33]:
response().download('real food images', 50)

In [47]:
response().download('landscape', 13)